In [4]:
import pandas as pd
import numpy as np

In [2]:
#create dataframe with original data that will not be touched throughout notebook
original_data = pd.read_csv('meteolytix_Artikelgruppen_Umsatz_verschiedeneStandorte.csv') 

#set Datum to datetime format
original_data['Datum'] = pd.to_datetime(original_data['Datum'])

#convert date to index
original_data.set_index('Datum', inplace=True)
original_data.sort_index(inplace=True)

#convert Filiale and Artikel ID to integer
original_data.Filiale = [int(no) for no in original_data.Filiale]
original_data.Artikelgruppe = [int(no) for no in original_data.Artikelgruppe]

In [9]:
#Split data into train, validation and test
train_df = original_data[original_data.index.year < 2018]
validation_df = original_data[original_data.index.year == 2018]
test_df = original_data[original_data.index.year == 2019]
corona_df = original_data[original_data.index.year == 2020]

#save dataframes as csv for later usage
for data in [train_df,validation_df,test_df,corona_df]:
    data.to_csv()

#function to extract data for only one store from dataframe and pivot the article column
def store_split(df, fil):
    #date_df = pd.DataFrame(index=pd.date_range(start=pd.to_datetime(str(df.index.min().year)+'-01-01'), end=df.index.max()))
    dummy = pd.pivot_table(df[df.Filiale == fil],values='Umsatz',index=df[df.Filiale == fil].index,columns='Artikelgruppe', aggfunc=np.sum, fill_value=0)
    dummy = dummy.reindex(pd.date_range(start=pd.to_datetime(str(df.index.min().year)+'-01-01'), end=df.index.max())).fillna(0)
    return dummy



In [ ]:
#iteratively generate individual train dataframes for each store
for fil in train_df.Filiale.unique():
    globals()['train_fil_'+str(fil)] = store_split(train_df,fil)
    globals()['train_fil_'+str(fil)].to_csv('data/train_fil_'+str(fil))
#iteratively generate individual train dataframes for each store
for fil in validation_df.Filiale.unique():
    globals()['validation_fil_'+str(fil)] = store_split(validation_df,fil)  
    globals()['validation_fil_'+str(fil)].to_csv('data/validation_fil_'+str(fil))
#iteratively generate individual train dataframes for each store
for fil in validation_df.Filiale.unique():
    globals()['test_fil_'+str(fil)] = store_split(test_df,fil)  
    globals()['test_fil_'+str(fil)].to_csv('data/test_fil_'+str(fil))